这个笔记本用来验证`lab_fea = select_feature('../../data/feature_chi.txt', max_feature)["1"]`中，select_feature这个函数的作用。下面是函数的整体部分：

In [1]:
import heapq 

def select_feature(filePath, k):
	read = open(filePath, 'r')
	lab_fea = {}
	
	for line in read:
		line_arr = line.strip().split()
		if len(line_arr) - 1 <= k:
			lab_fea[line_arr[0]] = [kv.split(':')[0] for kv in line_arr[1 : ]]
		else:
			heap = []
			heapq.heapify(heap)
			for kv in line_arr[1 : ]:
				key, val = kv.split(':')
				if len(heap) < k:
					heapq.heappush(heap, (float(val), key))
				else:
					if float(val) > heap[0][0]:
						heapq.heappop(heap)
						heapq.heappush(heap, (float(val), key))
			lab_fea[line_arr[0]] = [heapq.heappop(heap)[1] for i in range(len(heap))]
	read.close()
	return lab_fea

In [3]:
lab_fea = select_feature('feature_chi.txt', 1000)

In [12]:
read = open('feature_chi.txt', 'r')

In [19]:
num_lines = sum(1 for line in open('feature_chi.txt'))

In [20]:
num_lines

1

果然，那个文件实际只有一行。下面的for line in read也就没什么用处了，重要的是line

In [13]:
lab_fea = {}

In [21]:
for line in read:
    line_arr = line.strip().split()

In [23]:
line_arr[:10]

['1',
 'fawn:0.0',
 'tsukino:1.0',
 'saimin:1.0',
 'nunnery:1.0',
 'deferment:1.0',
 'sonja:0.0',
 'tilton:1.0',
 'gag:15.0',
 'woods:12.0']

In [24]:
len(line_arr)

74091

1000是我们选取的需要的feature单词量

In [25]:
k= 1000

In [26]:
if len(line_arr) - 1 <= k:
    for kv in line_arr[1:]: # 不包括一开始的‘1’
        lab_fea[line_arr[0]] = kv.split(':')[0]
else:
    heap = []
    heapq.heapify(heap)

heapq，这个模块(build-in)实现了一个堆的数据结构，完美的解决了Top-K问题，以后解决Top-K问题的时候，直接把这个模块拿来用就可以了

注意，默认的heap是一个小顶堆！

heapq.heapify(x) 将列表x进行堆调整，默认的是小顶堆。x必须是list，此函数将list变成堆，实时操作。从而能够在任何情况下使用堆的函数。

堆常用来实现优先队列，在这种队列中，待删除的元素为优先级最高（最低）的那个。在任何时候，任意优先元素都是可以插入到队列中去的，是计算机科学中一类特殊的数据结构的统称

最大（最小）堆是一棵每一个节点的键值都不小于（大于）其孩子（如果存在）的键值的树。大顶堆是一棵完全二叉树，同时也是一棵最大树。小顶堆是一棵完全完全二叉树，同时也是一棵最小树。

比如下图左边是最大堆，右边是最小堆。这里我们只用到了最小堆，即节点不大于其孩子即可，条件还算比较宽：

![](http://img.my.csdn.net/uploads/201210/16/1350354702_4619.jpg)

In [29]:
heap = []

In [30]:
heapq.heapify(heap)

In [31]:
heap

[]

In [34]:
key, val = line_arr[3].split(':')
print(key + ' and ' + val)

saimin and 1.0


heapq.heappush(heap, item) 把item添加到heap中（heap是一个列表）。

我看懂下面的目的了。一开始heap是小于1000的，所以先把前1000个`(0.0, 'abbott')`这样的tuple放入head中，对于1000个之后的元素，每一个元素的值先与堆顶的最小值比较（因为是最小堆，所以最小值都在堆顶），如果比最小值大的话，就把最小值pop出来，把新的值添加进入……感觉没什么神奇的地方，这么看的话，反倒是feature_chi.txt中每个单词对应的数值是如何计算的，反倒令人好奇。

In [35]:
for kv in line_arr[1:]:
    key, val = kv.split(':')
    if len(heap) < k:
        heapq.heappush(heap, (float(val), key))
    else:
        if float(val) > heap[0][0]:
            heapq.heappop(heap)
            heapq.heappush(heap, (float(val), key))

lab_fea[line_arr[0]] = [heapq.heappop(heap)[1] for i in range(len(heap))]


In [37]:
heap[:10]

[(0.0, 'abbott'),
 (0.0, 'adapt'),
 (0.0, 'absurdities'),
 (0.0, 'altogether'),
 (0.0, 'advisedly'),
 (0.0, 'academia'),
 (0.0, 'amfortas'),
 (0.0, 'cartooning'),
 (0.0, 'boarder'),
 (0.0, 'affiliated')]

因为这个lab_fea是用到TfidfVectorizer中，用来做vocabulary的，所以之后我可以直接去查一下如何制作TfidfVectorizer的vocabulary就行了。这样的话，可以直接把runBow.py也改写了。

In [ ]:
self.vectorizer = TfidfVectorizer(analyzer = "word",
                                 tokenizer = None,
                                 preprocessor = None,
                                 stop_words = None,
                                 vocabulary = lab_fea,
                                 max_features = max_feature)